In [1]:
%matplotlib inline
import numpy as np
import torch.nn as nn
import torch
import torch.optim as optim
torch.set_printoptions(edgeitems=2,linewidth=75)

In [2]:
t_c=[0.5, 14.0, 15.0, 28.0, 11.0, 8.0, 3.0, -4.0, 6.0, 13.0, 21.0]
t_u=[35.7, 55.9, 58.2, 81.9, 56.3, 28.9, 33.9, 21.8, 48.4, 60.4, 68.4]
t_c=torch.tensor(t_c).unsqueeze(1) # 1번 축에 여분의 차원을 추가
t_u=torch.tensor(t_u).unsqueeze(1)

t_u.shape

torch.Size([11, 1])

In [3]:
n_samples = t_u.shape[0]
n_val = int(0.2 * n_samples)

shuffled_indices = torch.randperm(n_samples) # 0~1 사이 임의의 정수 순열 반환

train_indices = shuffled_indices[:-n_val]
val_indices = shuffled_indices[-n_val:]

train_indices, val_indices

(tensor([6, 1, 8, 2, 0, 7, 5, 4, 3]), tensor([10,  9]))

In [4]:
t_u_train=t_u[train_indices]
t_c_train=t_c[train_indices]

t_u_val=t_u[val_indices]
t_c_val=t_c[val_indices]

t_un_train=0.1*t_u_train
t_un_val=0.1*t_u_val

In [5]:
linear_model=nn.Linear(1,1)
linear_model(t_un_val)

tensor([[3.1035],
        [2.6870]], grad_fn=<AddmmBackward0>)

In [6]:
linear_model.weight

Parameter containing:
tensor([[0.5206]], requires_grad=True)

In [7]:
linear_model.bias

Parameter containing:
tensor([-0.4575], requires_grad=True)

In [8]:
x=torch.ones(10,1)
linear_model(x) #하나의 입력을 받고 하나의 출력을 만듬

tensor([[0.0631],
        [0.0631],
        [0.0631],
        [0.0631],
        [0.0631],
        [0.0631],
        [0.0631],
        [0.0631],
        [0.0631],
        [0.0631]], grad_fn=<AddmmBackward0>)

In [9]:
optim=optim.SGD(
linear_model.parameters(),
lr=1e-2)

In [10]:
linear_model.parameters()

<generator object Module.parameters at 0x0000022C0CB8DE00>

In [11]:
list(linear_model.parameters())

[Parameter containing:
 tensor([[0.5206]], requires_grad=True),
 Parameter containing:
 tensor([-0.4575], requires_grad=True)]

In [12]:
def training_loop(n_epochs, optim, model, loss_fn, t_u_train, t_u_val,
                 t_c_train,t_c_val):
    for e in range(1,n_epochs+1):
        t_p_train=model(t_u_train)
        loss_train=loss_fn(t_p_train,t_c_train)
        
        t_p_val=model(t_u_val)
        loss_val=loss_fn(t_p_val,t_c_val)
        
        optim.zero_grad()
        loss_train.backward()
        optim.step()
        
        if e == 1 or e % 1000 ==0:
            print(f"Epoch {e}, Traning loss {loss_train.item():.4f},"
                  f" Validation loss{loss_val.item():.4f}")

In [13]:
training_loop(
    n_epochs=3000,
    optim=optim,
    model=linear_model,
    loss_fn=nn.MSELoss(),
    t_u_train=t_un_train,
    t_u_val=t_un_val,
    t_c_train=t_c_train,
    t_c_val=t_c_val)

print()
print(linear_model.weight)
print(linear_model.bias)

Epoch 1, Traning loss 115.0251, Validation loss213.3224
Epoch 1000, Traning loss 10.6855, Validation loss4.8569
Epoch 2000, Traning loss 10.5122, Validation loss4.6163
Epoch 3000, Traning loss 10.5107, Validation loss4.6001

Parameter containing:
tensor([[4.7217]], requires_grad=True)
Parameter containing:
tensor([-13.0302], requires_grad=True)


In [14]:
seq_model=nn.Sequential(
nn.Linear(1,13),
nn.Tanh(),
nn.Linear(13,1))

seq_model

Sequential(
  (0): Linear(in_features=1, out_features=13, bias=True)
  (1): Tanh()
  (2): Linear(in_features=13, out_features=1, bias=True)
)

In [15]:
for name, param in seq_model.named_parameters():
    print(name,param.shape)

0.weight torch.Size([13, 1])
0.bias torch.Size([13])
2.weight torch.Size([1, 13])
2.bias torch.Size([1])


In [16]:
from collections import OrderedDict

seq_model = nn.Sequential(OrderedDict([
    ('hidden_linear', nn.Linear(1, 8)),
    ('hidden_activation', nn.Tanh()),
    ('output_linear', nn.Linear(8, 1))
]))

seq_model

Sequential(
  (hidden_linear): Linear(in_features=1, out_features=8, bias=True)
  (hidden_activation): Tanh()
  (output_linear): Linear(in_features=8, out_features=1, bias=True)
)

In [17]:
for name, param in seq_model.named_parameters():
    print(name, param.shape)

hidden_linear.weight torch.Size([8, 1])
hidden_linear.bias torch.Size([8])
output_linear.weight torch.Size([1, 8])
output_linear.bias torch.Size([1])


In [18]:

seq_model.output_linear.bias

Parameter containing:
tensor([0.2560], requires_grad=True)

In [20]:
optim=optim.SGD(
seq_model.parameters(),
lr=1e-2)
training_loop(
    n_epochs = 5000, 
    optim = optim,
    model = seq_model,
    loss_fn = nn.MSELoss(),
    t_u_train = t_un_train,
    t_u_val = t_un_val, 
    t_c_train = t_c_train,
    t_c_val = t_c_val)
    
print('output', seq_model(t_un_val))
print('answer', t_c_val)
print('hidden', seq_model.hidden_linear.weight.grad)

AttributeError: 'SGD' object has no attribute 'SGD'

In [1]:
from matplotlib import pyplot as plt

t_range=torch.arange(20.,90.).unsqueeze(1)
fig=plt.figure(dpi=600)
plt.xlabel("Fahernheit")
plt.ylabel("Celsius")
plt.plot(t_u.numpy(),t_c.numpy(),'o')
plt.plot(t_range.numpy(),seq_model(0.1*t_range).detach().numpy(),'c-')
plt.plot(t_u.numpy(),seq.model(0.1*t_u).detach().numpy(),'kx')

NameError: name 'torch' is not defined